# Derivatives & Volatility — Analysis Notebook

This notebook demonstrates:
1. Black–Scholes pricing and implied volatility inversion
2. Historical volatility calculation
3. A simple Heston calibration on synthetic data


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.black_scholes import bs_price, implied_vol
from src.volatility import historical_volatility
from src.heston import heston_price, calibrate_heston

# Example: Black-Scholes price and implied vol
S = 100.0
K = 105.0
T = 30/252
r = 0.01
q = 0.0
sigma = 0.22
call_price = bs_price(S,K,T,r,q,sigma,'call')
iv = implied_vol(call_price, S, K, T, r, q, 'call')
print('BS call price:', call_price)
print('Recovered implied vol:', iv)

# Historical volatility (synthetic prices)
dates = pd.date_range('2020-01-01', periods=252)
np.random.seed(42)
rets = np.random.normal(0, 0.01, size=len(dates))
prices = pd.Series(100.0 * np.exp(np.cumsum(rets)), index=dates)
hv = historical_volatility(prices, window=21)
hv.tail()

# Simple Heston calibration on synthetic market
true_params = (1.5, 0.04, 0.4, -0.5, 0.04)
strikes = [80,90,100,110,120]
market = []
for K in strikes:
    p = heston_price(S, K, T, r, q, *true_params)
    market.append({'strike': K, 'T': T, 'option_type': 'call', 'market_price': p})
init = (2.0, 0.02, 0.3, -0.3, 0.02)
res = calibrate_heston(market, S, r, q, init)
print('Calibration result:')
print(res)
